In [14]:
!pip install -q langchain langchain-nvidia-ai-endpoints duckduckgo-search langchain-groq langchain-community langchain-core chromadb pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 31.7 MB/s eta 0:00:00


In [47]:
import os

from google.colab import userdata
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
# os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [75]:
# Required libraries

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.runnables import RunnableLambda
from langchain_groq import ChatGroq
from langchain.utilities import DuckDuckGoSearchAPIWrapper

In [76]:
# Few Shot Examples
examples = [

    {
        "input": "What is the birthplace of Albert Einstein?",
        "output": "what is Albert Einstein's personal history?",
    },
    {
        "input": "Can a Tesla car drive itself?",
        "output": "what can a Tesla car do?",
    },
    {
        "input": "Did Queen Elizabeth II ever visit Canada?",
        "output": "what is Queen Elizabeth II's travel history?",
    },
    {
        "input": "Can a SpaceX rocket land itself?",
        "output": "what can a SpaceX rocket do?",
    }
]

In [77]:
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ('human','{input}'),
        ('ai','{output}')
    ]
)

In [78]:
# using few shot prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [79]:
# prompt
system = """You are expert at world knowledge. your task is to step back paraphrase question to a more genric step-back question, which is easier to answer. Here are few examples.
"""

prompt = ChatPromptTemplate.from_messages(
    [
      ('system',system),
      few_shot_prompt,
      ('user','{question}')

    ]
)

In [80]:
# llm model
# llm = ChatNVIDIA(model='google/gemma-2-9b-it')
llm = ChatGroq(model_name='llama-3.1-70b-versatile')

question_gen = prompt | llm | StrOutputParser()

In [81]:
question = "was chatgpt around while trump was president?"

question_gen.invoke({"question": question})

'when was ChatGPT developed?'

In [84]:
search = DuckDuckGoSearchAPIWrapper(max_results=4)

def retriever(query):
  return search.run(query=query)

In [87]:
# Response prompt
template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
{normal_context}
{step_back_context}

Original question: {question}

Answer:
"""

response_prompt = ChatPromptTemplate.from_template(template)

In [88]:
chain = ({
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
    "step_back_context": question_gen | retriever,
    "question": lambda x: x["question"]
          }
         | response_prompt
         | llm
         | StrOutputParser()
    )

In [89]:
question = "was chatgpt around while trump was president?"

chain.invoke({"question": question})

"Based on the provided context, ChatGPT was not available during Donald Trump's presidency. According to the given information, ChatGPT was released on November 30, 2022, and Donald Trump's presidency ended on January 20, 2021. Therefore, there was a nearly two-year gap between the end of Trump's presidency and the release of ChatGPT."

In [91]:
question2 = 'how to learn ML prepare a roadmap'

chain.invoke({"question": question2})

"To learn Machine Learning (ML) from scratch, I've prepared a comprehensive roadmap to guide you through the process. This roadmap will cover the essential steps, skills, and resources needed to become proficient in ML.\n\n**Phase 1: Foundational Courses (3-6 months)**\n\n1. **Mathematics**:\n\t* Linear Algebra: Understand vectors, matrices, and operations.\n\t* Calculus: Familiarize yourself with derivatives and optimization techniques.\n\t* Probability and Statistics: Study probability distributions, Bayes' theorem, and statistical inference.\n\t* Resources: Khan Academy, MIT OpenCourseWare, and Coursera.\n2. **Programming**:\n\t* Python: Focus on Python 3.x, as it's the primary language used in ML.\n\t* Familiarize yourself with popular libraries: NumPy, pandas, and scikit-learn.\n\t* Practice with projects on platforms like LeetCode, HackerRank, or Codewars.\n3. **Data Structures and Algorithms**:\n\t* Study data structures: arrays, linked lists, stacks, queues, trees, and graphs.\